In [1]:
import json
import requests
from datetime import datetime
from collections import defaultdict
from time import sleep

#https://pypi.org/project/gamma-viewer/
# from gamma_viewer import GammaViewer
from IPython.display import display

# from bmt import Toolkit
# t = Toolkit()

In [2]:
def log_query(query, ars_url, pk, time=datetime.now()):
    message = f'''
{json.dumps(query, indent=4)}
{ars_url}
{pk}
{time}
'''
    try:
        with open('log.txt', 'a') as fh:
            fh.write(message)
    except:
        print('Could not write to log file')
        
def submit_to_ars(m,ars_url='https://ars.transltr.io/ars/api',arax_url='https://arax.ncats.io'):
    submit_url=f'{ars_url}/submit'
    response = requests.post(submit_url,json=m)
    try:
        message_id = response.json()['pk']
    except:
        print('fail')
        message_id = None
    print(f'{arax_url}/?source=ARS&id={message_id}')
    
    t = datetime.now()
    print(f'Submitted at {t} to {ars_url}')
    log_query(m, ars_url, message_id, t)
    
    return message_id

def retrieve_ars_results(mid,ars_url='https://ars.transltr.io/ars/api'):
    message_url = f'{ars_url}/messages/{mid}?trace=y'
    response = requests.get(message_url)
    j = response.json()
    print( j['status'] )
    results = {}
    for child in j['children']:
        print(child['status'])
        if child['status']  == 'Done':
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                nresults = len(child_response['fields']['data']['message']['results'])
                if nresults > 0:
                    results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                nresults=0
                child['status'] = 'ARS Error'
        elif child['status'] == 'Error':
            nresults=0
            childmessage_id = child['message']
            child_url = f'{ars_url}/messages/{childmessage_id}'
            try:
                child_response = requests.get(child_url).json()
                results[child['actor']['agent']] = {'message':child_response['fields']['data']['message']}
            except Exception as e:
                print(e)
                child['status'] = 'ARS Error'
        else:
            nresults = 0
        print( child['status'], child['actor']['agent'],nresults )
    return results

def submit_to_devars(m):
    return submit_to_ars(m,ars_url='https://ars-dev.transltr.io/ars/api',arax_url='https://arax.ncats.io')

def retrieve_devars_results(m):
    return retrieve_ars_results(m,ars_url='https://ars-dev.transltr.io/ars/api')

def printjson(j):
    print(json.dumps(j,indent=4))

def print_json(j):
    printjson(j)
    
def name_lookup(text):
    url= f'https://name-resolution-sri.renci.org/lookup?string={text}&offset=0&limit=10'
    response = requests.post(url)
    printjson(response.json())
    
def one_hop_message(subject_category,object_category,predicate=None,subject_id = None, object_id = None):
    query_graph = {
    "nodes": {
        'a':{
            "category": subject_category
        },
        'b':{
            "category": object_category
        }
    },
    "edges": {
        'ab':{
            "subject": "a",
            "object": "b",
            #"predicate": predicate
        }
    }
    }
    if predicate is not None:
        query_graph['edges']['ab']['predicate'] = predicate
    if subject_id is not None:
        query_graph['nodes']['a']['id'] = subject_id
    if object_id is not None:
        query_graph['nodes']['b']['id'] = object_id
    message = {"message": {"query_graph": query_graph},'knowledge_graph':{"nodes": [], "edges": [],}, 'results':[]}
    return message

def send_trapi(m, service_name=None, service_url=None):
    services = {
        'rtx-kg2': 'https://arax.ncats.io/api/rtxkg2/v1.2/query',
        'icees-asthma': 'https://icees.renci.org:16339/query?reasoner=true&verbose=false',
        'icees-dili': 'https://icees.renci.org:16341/query?reasoner=true&verbose=false',
        'arax': 'https://arax.ncats.io/api/arax/v1.2/query'
    }
    
    if service_url is None and service_name is not None:
        service_url = services.get(service_name.lower())
        
    if service_url is None:
        print('No service URL')
       
    t1 = datetime.now()
    print(f'Submitting at {t1} to {service_url}')
    response = requests.post(service_url,json=m)
    try:
        j = response.json()
    except:
        print('fail')
        j = None    
    
    t2 = datetime.now()    
    print(f'Query took {(t2-t1).total_seconds()} seconds')
    
    return j

def rtx_result_summary(response):
    print(response['description'])
    n_results = len(response['message']['results'])
    print(f"Number of results: {n_results}")
    if n_results:
        print(f"Number of nodes bound to n2: {len(response['message']['results'][0]['node_bindings']['n2'])}")
        print(f"Number of nodes bound to n3: {len(response['message']['results'][0]['node_bindings']['n3'])}")
        
def look_for_desired_answers(response):
    looking_for=['resveratrol', 'prednisone', 'dexamethasone', 'silybin']
    for x in response['message']['results'][0]['node_bindings']['n3']:
        name = response['message']['knowledge_graph']['nodes'][x['id']]['name']
        for desired in looking_for:
            if desired in name:
                print(name)

# CURIEs
    
    primary ciliary dyskinesia (MONDO:0016575)
    cystic fibrosis (MONDO:0009061)
    idiopathic bronchiectasis (MONDO:0018956)
    lymphangioleiomyomatosis (MONDO:0011705)
    idiopathic pulmonary fibrosis (MONDO:0008345)

    Related CURIES: disorder of lung (SCTID: 19829001), lower respiratory tract disorder (MONDO:0000270)


# (rare pulmonary disease) --[has_real_world_evidence_of_association_with]-- (ChemicalEntity)

In [3]:
submit_to_ars(json.loads(
"""
{
    "message": {
        "query_graph": {
            "nodes": {
                "n0": {
                    "ids": ["MONDO:0016575"],
                    "categories": [
                        "biolink:DiseaseOrPhenotypicFeature"
                    ],
                    "name": "primary ciliary dyskinesia"
                },
                "n1": {
                    "categories": [
                        "biolink:ChemicalEntity"
                    ]
                }
            },
            "edges": {
                "e0": {
                    "subject": "n0",
                    "object": "n1",
                    "predicates": ["biolink:has_real_world_evidence_of_association_with"]
                }
            }
        }
    }
}
"""
))

https://arax.ncats.io/?source=ARS&id=bdf4d112-c02c-477e-bd09-e17c21a1ac99
Submitted at 2022-05-04 09:51:46.907441 to https://ars.transltr.io/ars/api


'bdf4d112-c02c-477e-bd09-e17c21a1ac99'

https://arax.ncats.io/?source=ARS&id=bdf4d112-c02c-477e-bd09-e17c21a1ac99  
Submitted at 2022-05-04 09:51:46.907441 to https://ars.transltr.io/ars/api  

In [7]:
submit_to_ars(json.loads(
"""
{
    "message": {
        "query_graph": {
            "nodes": {
                "n0": {
                    "ids": ["MONDO:0009061"],
                    "categories": [
                        "biolink:DiseaseOrPhenotypicFeature"
                    ],
                    "name": "cystic fibrosis"
                },
                "n1": {
                    "categories": [
                        "biolink:ChemicalEntity"
                    ]
                }
            },
            "edges": {
                "e0": {
                    "subject": "n0",
                    "object": "n1",
                    "predicates": ["biolink:has_real_world_evidence_of_association_with"]
                }
            }
        }
    }
}
"""
))

https://arax.ncats.io/?source=ARS&id=daf75a37-3e1b-4e6e-9f1c-fcd3b5ef43ac
Submitted at 2022-05-04 10:00:06.165670 to https://ars.transltr.io/ars/api


'daf75a37-3e1b-4e6e-9f1c-fcd3b5ef43ac'

https://arax.ncats.io/?source=ARS&id=daf75a37-3e1b-4e6e-9f1c-fcd3b5ef43ac  
Submitted at 2022-05-04 10:00:06.165670 to https://ars.transltr.io/ars/api  

In [6]:
submit_to_ars(json.loads(
"""
{
    "message": {
        "query_graph": {
            "nodes": {
                "n0": {
                    "ids": ["MONDO:0018956"],
                    "categories": [
                        "biolink:DiseaseOrPhenotypicFeature"
                    ],
                    "name": "idiopathic bronchiectasis"
                },
                "n1": {
                    "categories": [
                        "biolink:ChemicalEntity"
                    ]
                }
            },
            "edges": {
                "e0": {
                    "subject": "n0",
                    "object": "n1",
                    "predicates": ["biolink:has_real_world_evidence_of_association_with"]
                }
            }
        }
    }
}
"""
))

https://arax.ncats.io/?source=ARS&id=4067b5cc-31c2-4fd9-a7ab-1c36983a036a
Submitted at 2022-05-04 09:59:24.271224 to https://ars.transltr.io/ars/api


'4067b5cc-31c2-4fd9-a7ab-1c36983a036a'

https://arax.ncats.io/?source=ARS&id=4067b5cc-31c2-4fd9-a7ab-1c36983a036a  
Submitted at 2022-05-04 09:59:24.271224 to https://ars.transltr.io/ars/api  

In [5]:
submit_to_ars(json.loads(
"""
{
    "message": {
        "query_graph": {
            "nodes": {
                "n0": {
                    "ids": ["MONDO:0011705"],
                    "categories": [
                        "biolink:DiseaseOrPhenotypicFeature"
                    ],
                    "name": "lymphangioleiomyomatosis"
                },
                "n1": {
                    "categories": [
                        "biolink:ChemicalEntity"
                    ]
                }
            },
            "edges": {
                "e0": {
                    "subject": "n0",
                    "object": "n1",
                    "predicates": ["biolink:has_real_world_evidence_of_association_with"]
                }
            }
        }
    }
}
"""
))

https://arax.ncats.io/?source=ARS&id=d10a44c9-4185-4eca-a236-067b6fa77b49
Submitted at 2022-05-04 09:58:41.820329 to https://ars.transltr.io/ars/api


'd10a44c9-4185-4eca-a236-067b6fa77b49'

https://arax.ncats.io/?source=ARS&id=d10a44c9-4185-4eca-a236-067b6fa77b49  
Submitted at 2022-05-04 09:58:41.820329 to https://ars.transltr.io/ars/api  

In [4]:
submit_to_ars(json.loads(
"""
{
    "message": {
        "query_graph": {
            "nodes": {
                "n0": {
                    "ids": ["MONDO:0008345"],
                    "categories": [
                        "biolink:DiseaseOrPhenotypicFeature"
                    ],
                    "name": "idiopathic pulmonary fibrosis"
                },
                "n1": {
                    "categories": [
                        "biolink:ChemicalEntity"
                    ]
                }
            },
            "edges": {
                "e0": {
                    "subject": "n0",
                    "object": "n1",
                    "predicates": ["biolink:has_real_world_evidence_of_association_with"]
                }
            }
        }
    }
}
"""
))

https://arax.ncats.io/?source=ARS&id=bfcacde4-ea07-4ae3-b53f-a97f39ab10c6
Submitted at 2022-05-04 09:57:38.646776 to https://ars.transltr.io/ars/api


'bfcacde4-ea07-4ae3-b53f-a97f39ab10c6'

https://arax.ncats.io/?source=ARS&id=bfcacde4-ea07-4ae3-b53f-a97f39ab10c6  
Submitted at 2022-05-04 09:57:38.646776 to https://ars.transltr.io/ars/api  
 